# 🏆 March Machine: NCAA Predictions - End to End Aproach
Hello, the purpose of this notebook is to build a simple yet powerful model. My goal is to create straightforward functions that are accessible to beginners while still providing value for advanced users.

I created this notebook nearly three years ago and plan to use it as a foundation for significant improvements. It's remarkable how time flies.

https://www.kaggle.com/code/cv13j0/ncaa-gradient-boosted-trees-xgboost

## 1.0 Loading Default Libraries.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

---

## 2.0 Loading the Competition Information.

In [2]:
def load_csv_to_dataframe(file_path, ignore_fields=[]):
    """
    Load a CSV file into a pandas DataFrame, optionally ignoring specified fields.

    Parameters:
    file_path (str): The file path of the CSV file to be loaded.
    ignore_fields (list): A list of field names to be ignored when loading the CSV.

    Returns:
    pandas.DataFrame: A DataFrame containing the data from the CSV file, excluding the ignored fields.
    """
    # Read the CSV file from the given file path using pandas
    df = pd.read_csv(file_path)
    
    # Drop the fields that need to be ignored, if they exist in the DataFrame
    df = df.drop(columns=ignore_fields, errors='ignore')
    
    # Return the resulting DataFrame
    return df

# Example usage:
# df = load_csv_to_dataframe('data/sample.csv', ignore_fields=['column_to_ignore'])
# print(df.head())

In [3]:
# Load the competiion dataset
MRegularSeasonCompactResults = '/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv'
WRegularSeasonCompactResults = '/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv'
SampleSubmissionStage1 = '/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage1.csv' 

#original_input = '/kaggle/input/student-bag-price-prediction-dataset/Noisy_Student_Bag_Price_Prediction_Dataset.csv'

MRegularSeasonCompactResults_df = load_csv_to_dataframe(MRegularSeasonCompactResults, ignore_fields=['id'])
WRegularSeasonCompactResults_df = load_csv_to_dataframe(WRegularSeasonCompactResults, ignore_fields=['id'])
SampleSubmissionStage1_df = load_csv_to_dataframe(SampleSubmissionStage1)

In [4]:
MRegularSeasonCompactResults_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190771 entries, 0 to 190770
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Season   190771 non-null  int64 
 1   DayNum   190771 non-null  int64 
 2   WTeamID  190771 non-null  int64 
 3   WScore   190771 non-null  int64 
 4   LTeamID  190771 non-null  int64 
 5   LScore   190771 non-null  int64 
 6   WLoc     190771 non-null  object
 7   NumOT    190771 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 11.6+ MB


In [5]:
MRegularSeasonCompactResults_df.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
190766,2025,85,1401,75,1328,68,H,0
190767,2025,85,1405,86,1325,83,A,0
190768,2025,85,1428,69,1153,66,H,0
190769,2025,85,1444,74,1123,71,A,0
190770,2025,85,1461,83,1201,72,H,1


In [6]:
WRegularSeasonCompactResults_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134961 entries, 0 to 134960
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Season   134961 non-null  int64 
 1   DayNum   134961 non-null  int64 
 2   WTeamID  134961 non-null  int64 
 3   WScore   134961 non-null  int64 
 4   LTeamID  134961 non-null  int64 
 5   LScore   134961 non-null  int64 
 6   WLoc     134961 non-null  object
 7   NumOT    134961 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 8.2+ MB


In [7]:
WRegularSeasonCompactResults_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


In [8]:
SampleSubmissionStage1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507108 entries, 0 to 507107
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      507108 non-null  object 
 1   Pred    507108 non-null  float64
dtypes: float64(1), object(1)
memory usage: 7.7+ MB


In [9]:
SampleSubmissionStage1_df.head()

,ID,Pred
0,2021_1101_1102,0.5
1,2021_1101_1103,0.5
2,2021_1101_1104,0.5
3,2021_1101_1105,0.5
4,2021_1101_1106,0.5
